介绍如何在pytorch环境下，使用FGSM算法攻击基于MNIST数据集预训练的CNN/MLP模型。运行该文件前，需要先运行指定文件生成对应的模型：

    cd AdvBox/tutorials
    python mnist_model_pytorch.py
     

Jupyter notebook中使用Anaconda中的环境需要单独配置，默认情况下使用的是系统默认的Python环境，以使用advbox环境为例。
首先在默认系统环境下执行以下命令，安装ipykernel。

    conda install ipykernel
    conda install -n advbox ipykernel

在advbox环境下激活，这样启动后就可以在界面上看到advbox了。

    python -m ipykernel install --user --name advbox --display-name advbox 


In [1]:
#调试开关
import logging
#logging.basicConfig(level=logging.INFO,format="%(filename)s[line:%(lineno)d] %(levelname)s %(message)s")
#logger=logging.getLogger(__name__)
import sys

#添加系统路径
sys.path.append("../AdvBox/")

import torch
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data.dataloader as Data
from advbox.adversary import Adversary
from advbox.attacks.gradient_method import FGSM
from advbox.models.pytorch import PytorchModel
from tutorials.mnist_model_pytorch import Net

train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])


In [2]:
TOTAL_NUM = 10000
pretrained_model="../AdvBox/tutorials/mnist-pytorch/net.pth"
loss_func = torch.nn.CrossEntropyLoss()

#使用MNIST测试数据集 随机挑选TOTAL_NUM个
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../AdvBox/tutorials/mnist-pytorch/data', train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
    ])),
    batch_size=1, shuffle=True)

# Define what device we are using
logging.info("CUDA Available: {}".format(torch.cuda.is_available()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = Net().to(device)

# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

# advbox demo
m = PytorchModel(
    model, loss_func,(0, 1),
    channel_axis=1)

#实例化FGSM
attack = FGSM(m)
#设置攻击步长为0.1
attack_config = {"epsilons": 0.1}

# use test data to generate adversarial examples
total_count = 0
fooling_count = 0

for i, data in enumerate(test_loader):
    inputs, labels = data
    inputs, labels=inputs.numpy(),labels.numpy()

    total_count += 1
    adversary = Adversary(inputs, labels[0])

    # FGSM non-targeted attack
    adversary = attack(adversary, **attack_config)

    if adversary.is_successful():
        fooling_count += 1
        print(
            'attack success, original_label=%d, adversarial_label=%d, count=%d'
            % (labels, adversary.adversarial_label, total_count))

    else:
        print('attack failed, original_label=%d, count=%d' %
              (labels, total_count))

    if total_count >= TOTAL_NUM:
        print(
            "[TEST_DATASET]: fooling_count=%d, total_count=%d, fooling_rate=%f"
            % (fooling_count, total_count,
               float(fooling_count) / total_count))
        break
print("fgsm attack done")


cuda
attack success, original_label=2, adversarial_label=7, count=1
attack success, original_label=4, adversarial_label=7, count=2
attack success, original_label=2, adversarial_label=8, count=3
attack success, original_label=3, adversarial_label=5, count=4
attack success, original_label=8, adversarial_label=2, count=5
attack success, original_label=7, adversarial_label=2, count=6
attack success, original_label=2, adversarial_label=7, count=7
attack success, original_label=1, adversarial_label=7, count=8
attack success, original_label=3, adversarial_label=0, count=9
attack success, original_label=9, adversarial_label=4, count=10
attack success, original_label=8, adversarial_label=9, count=11
attack success, original_label=4, adversarial_label=9, count=12
attack success, original_label=1, adversarial_label=7, count=13
attack success, original_label=5, adversarial_label=8, count=14
attack success, original_label=0, adversarial_label=6, count=15
attack success, original_label=6, adversaria

attack success, original_label=9, adversarial_label=4, count=147
attack success, original_label=5, adversarial_label=8, count=148
attack success, original_label=7, adversarial_label=2, count=149
attack success, original_label=7, adversarial_label=9, count=150
attack success, original_label=7, adversarial_label=3, count=151
attack success, original_label=1, adversarial_label=4, count=152
attack success, original_label=2, adversarial_label=7, count=153
attack success, original_label=4, adversarial_label=9, count=154
attack success, original_label=8, adversarial_label=0, count=155
attack success, original_label=6, adversarial_label=5, count=156
attack success, original_label=5, adversarial_label=8, count=157
attack success, original_label=9, adversarial_label=4, count=158
attack success, original_label=4, adversarial_label=9, count=159
attack success, original_label=0, adversarial_label=2, count=160
attack success, original_label=2, adversarial_label=4, count=161
attack success, original_

attack success, original_label=5, adversarial_label=3, count=280
attack success, original_label=8, adversarial_label=5, count=281
attack success, original_label=0, adversarial_label=2, count=282
attack success, original_label=5, adversarial_label=8, count=283
attack success, original_label=6, adversarial_label=5, count=284
attack success, original_label=9, adversarial_label=4, count=285
attack success, original_label=6, adversarial_label=4, count=286
attack success, original_label=2, adversarial_label=0, count=287
attack success, original_label=8, adversarial_label=2, count=288
attack success, original_label=5, adversarial_label=8, count=289
attack success, original_label=4, adversarial_label=8, count=290
attack success, original_label=4, adversarial_label=9, count=291
attack success, original_label=1, adversarial_label=4, count=292
attack success, original_label=6, adversarial_label=5, count=293
attack success, original_label=6, adversarial_label=5, count=294
attack success, original_

attack success, original_label=1, adversarial_label=4, count=414
attack success, original_label=3, adversarial_label=5, count=415
attack success, original_label=0, adversarial_label=9, count=416
attack success, original_label=6, adversarial_label=4, count=417
attack success, original_label=1, adversarial_label=7, count=418
attack success, original_label=5, adversarial_label=9, count=419
attack success, original_label=1, adversarial_label=4, count=420
attack success, original_label=6, adversarial_label=5, count=421
attack success, original_label=4, adversarial_label=7, count=422
attack success, original_label=9, adversarial_label=4, count=423
attack success, original_label=6, adversarial_label=0, count=424
attack success, original_label=8, adversarial_label=6, count=425
attack success, original_label=4, adversarial_label=9, count=426
attack success, original_label=3, adversarial_label=5, count=427
attack success, original_label=0, adversarial_label=6, count=428
attack success, original_

attack success, original_label=2, adversarial_label=0, count=546
attack success, original_label=0, adversarial_label=5, count=547
attack success, original_label=2, adversarial_label=8, count=548
attack success, original_label=9, adversarial_label=4, count=549
attack success, original_label=9, adversarial_label=4, count=550
attack success, original_label=6, adversarial_label=0, count=551
attack success, original_label=3, adversarial_label=5, count=552
attack success, original_label=5, adversarial_label=9, count=553
attack success, original_label=9, adversarial_label=7, count=554
attack success, original_label=3, adversarial_label=8, count=555
attack success, original_label=3, adversarial_label=5, count=556
attack success, original_label=4, adversarial_label=7, count=557
attack success, original_label=3, adversarial_label=5, count=558
attack success, original_label=4, adversarial_label=7, count=559
attack success, original_label=7, adversarial_label=8, count=560
attack success, original_

...

attack success, original_label=1, adversarial_label=4, count=9385
attack success, original_label=9, adversarial_label=4, count=9386
attack success, original_label=4, adversarial_label=7, count=9387
attack success, original_label=2, adversarial_label=7, count=9388
attack success, original_label=5, adversarial_label=8, count=9389
attack success, original_label=5, adversarial_label=8, count=9390
attack success, original_label=7, adversarial_label=2, count=9391
attack success, original_label=1, adversarial_label=4, count=9392
attack success, original_label=7, adversarial_label=9, count=9393
attack success, original_label=8, adversarial_label=9, count=9394
attack success, original_label=4, adversarial_label=9, count=9395
attack success, original_label=6, adversarial_label=8, count=9396
attack success, original_label=7, adversarial_label=2, count=9397
attack success, original_label=1, adversarial_label=4, count=9398
attack success, original_label=5, adversarial_label=8, count=9399
attack suc

attack success, original_label=2, adversarial_label=0, count=9516
attack success, original_label=3, adversarial_label=5, count=9517
attack success, original_label=2, adversarial_label=3, count=9518
attack success, original_label=6, adversarial_label=5, count=9519
attack success, original_label=1, adversarial_label=4, count=9520
attack success, original_label=0, adversarial_label=9, count=9521
attack success, original_label=2, adversarial_label=8, count=9522
attack success, original_label=1, adversarial_label=7, count=9523
attack success, original_label=0, adversarial_label=5, count=9524
attack success, original_label=2, adversarial_label=1, count=9525
attack success, original_label=9, adversarial_label=4, count=9526
attack success, original_label=5, adversarial_label=8, count=9527
attack success, original_label=3, adversarial_label=5, count=9528
attack success, original_label=9, adversarial_label=4, count=9529
attack success, original_label=5, adversarial_label=3, count=9530
attack suc

attack success, original_label=6, adversarial_label=4, count=9651
attack success, original_label=1, adversarial_label=4, count=9652
attack success, original_label=6, adversarial_label=5, count=9653
attack success, original_label=7, adversarial_label=9, count=9654
attack success, original_label=8, adversarial_label=3, count=9655
attack success, original_label=9, adversarial_label=7, count=9656
attack success, original_label=2, adversarial_label=0, count=9657
attack success, original_label=7, adversarial_label=1, count=9658
attack success, original_label=0, adversarial_label=8, count=9659
attack success, original_label=2, adversarial_label=1, count=9660
attack success, original_label=3, adversarial_label=8, count=9661
attack success, original_label=1, adversarial_label=8, count=9662
attack success, original_label=4, adversarial_label=9, count=9663
attack success, original_label=0, adversarial_label=2, count=9664
attack success, original_label=0, adversarial_label=9, count=9665
attack suc

attack success, original_label=6, adversarial_label=0, count=9782
attack success, original_label=9, adversarial_label=4, count=9783
attack success, original_label=9, adversarial_label=4, count=9784
attack success, original_label=9, adversarial_label=4, count=9785
attack success, original_label=8, adversarial_label=5, count=9786
attack success, original_label=1, adversarial_label=4, count=9787
attack success, original_label=1, adversarial_label=4, count=9788
attack success, original_label=2, adversarial_label=4, count=9789
attack success, original_label=7, adversarial_label=3, count=9790
attack success, original_label=2, adversarial_label=7, count=9791
attack success, original_label=6, adversarial_label=0, count=9792
attack success, original_label=0, adversarial_label=6, count=9793
attack success, original_label=1, adversarial_label=7, count=9794
attack success, original_label=3, adversarial_label=5, count=9795
attack success, original_label=7, adversarial_label=4, count=9796
attack suc

attack success, original_label=9, adversarial_label=4, count=9914
attack success, original_label=9, adversarial_label=4, count=9915
attack success, original_label=1, adversarial_label=4, count=9916
attack success, original_label=8, adversarial_label=5, count=9917
attack success, original_label=4, adversarial_label=9, count=9918
attack success, original_label=9, adversarial_label=4, count=9919
attack success, original_label=3, adversarial_label=2, count=9920
attack success, original_label=6, adversarial_label=5, count=9921
attack success, original_label=6, adversarial_label=4, count=9922
attack success, original_label=1, adversarial_label=4, count=9923
attack success, original_label=4, adversarial_label=9, count=9924
attack success, original_label=3, adversarial_label=7, count=9925
attack success, original_label=8, adversarial_label=7, count=9926
attack success, original_label=1, adversarial_label=4, count=9927
attack success, original_label=3, adversarial_label=8, count=9928
attack suc